# * Ex-T Revenue & Subs

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    WHERE FLAG = 1
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-20, 09:53:03

   -> Summany DataFrame : 427 rows, 17 columns

TDMDBPR : Disconnected


In [3]:
''' Current Month Revenue '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-19 04:44:12,20250301,20250316,0,0,"1,332,304,559","1,332,298,669","1,332,298,669","1,326,661,283",0,5,1104,17664
1,TB2R000100,Postpaid Revenue,2025-03-19 04:20:10,20250302,20250316,0,"2,160,837,719","2,160,563,270","2,160,561,516","2,160,561,516",0,0,5,177,885
2,TB2R010100,Postpaid Revenue B2C,2025-03-19 04:21:08,20250302,20250316,0,"1,924,545,236","1,924,349,244","1,924,347,490","1,924,347,490","1,918,113,364",0,6,1105,5525
3,TB2R020100,Postpaid Revenue B2B,2025-03-19 04:20:45,20250302,20250316,0,"236,292,483","236,214,026","236,214,026","236,214,026",0,0,5,177,885
4,TB3R000100,Broadband Revenue,2025-03-19 05:10:10,20250302,20250316,0,0,"938,881,327","938,881,327","938,881,327","938,935,299",0,5,1102,5507
5,TB4R000100,TVS Revenue,2025-03-19 14:24:58,20250302,20250316,0,0,"117,600,632","117,877,263","117,925,721",0,0,4,176,880


In [4]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-10 04:44:49,20240101,20250228,0,0,"2,300,952,636","2,331,094,199","2,331,094,199","2,321,368,786",0,5,1104,469200
1,TB2R000100,Postpaid Revenue,2025-03-18 21:52:19,20240102,20250228,0,"3,702,992,412","3,649,426,589","3,673,091,233","3,673,091,233",0,0,5,177,24603
2,TB2R010100,Postpaid Revenue B2C,2025-03-18 21:52:58,20240102,20250228,0,"3,294,112,826","3,265,094,596","3,287,264,358","3,287,265,328","3,263,477,002",0,6,1105,148232
3,TB2R020100,Postpaid Revenue B2B,2025-03-18 21:52:45,20240102,20250228,0,"408,879,586","384,331,993","385,826,874","385,826,874",0,0,5,177,24603
4,TB3R000100,Broadband Revenue,2025-03-07 05:10:02,20240102,20250228,0,0,"1,573,227,163","1,573,227,163","1,573,196,204","1,560,922,362",0,5,1102,138718
5,TB4R000100,TVS Revenue,2025-03-14 13:59:20,20240102,20250228,0,0,"328,867,343","323,172,106","327,520,964","322,352,854",0,5,1104,106301


In [5]:
''' Current Month Subscription '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-19 04:50:14,20250317,20250317,0,0,"17,500,348","17,500,348","17,500,348","17,438,408",0,5,1104,1104
1,TB1S000600,Prepay Active Caller Subs,2025-03-16 04:50:51,20250314,20250314,0,"14,567,698","14,355,888","14,355,888","14,355,888","14,304,384",0,6,1105,1105
2,TB2S000500,Postpaid Active Subs,2025-03-19 05:00:26,20250318,20250318,0,"10,199,076","10,198,531","10,198,519","10,198,519","10,123,400",0,6,1105,1105
3,TB2S010500,Postpaid Active Subs B2C,2025-03-19 05:01:34,20250318,20250318,0,"9,017,424","9,017,177","9,017,165","9,017,165","8,961,887",0,6,1105,1105
4,TB2S020400,Postpaid Active Subs B2B,2025-03-19 04:59:31,20250318,20250318,0,"1,181,652","1,181,354","1,181,354","1,181,354",0,0,5,177,177
5,TB3S000500,Broadband Active Subs,2025-03-19 04:55:27,20250318,20250318,0,0,"3,121,948","3,121,948","3,121,948","3,121,948",0,5,1102,1102
6,TB4S000500,TVS Active Subs,2025-03-19 04:55:38,20250318,20250318,0,0,"1,156,883","1,156,883","1,156,883","1,156,868",0,5,1104,1104


In [6]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-03 04:50:27,20240131,20250228,0,0,"17,638,410","17,652,566","17,652,566","17,591,038",0,5,1104,15456
1,TB1S000600,Prepay Active Caller Subs,2025-03-07 04:50:40,20240131,20250228,0,"14,441,024","14,248,564","14,362,024","14,362,051","14,312,460",0,6,1105,15470
2,TB2S000500,Postpaid Active Subs,2025-03-10 04:31:27,20240131,20250228,0,"10,021,684","9,862,550","9,923,667","9,923,692","9,851,246",0,6,1105,15470
3,TB2S010500,Postpaid Active Subs B2C,2025-03-10 04:31:57,20240131,20250228,0,"8,891,594","8,828,202","8,883,744","8,883,768","8,212,087",0,6,1105,15470
4,TB2S020400,Postpaid Active Subs B2B,2025-03-04 02:37:03,20240131,20250228,0,"1,130,090","1,034,348","1,039,923","1,039,923",0,0,5,177,2478
5,TB3S000500,Broadband Active Subs,2025-03-01 04:55:28,20240131,20250228,0,0,"3,053,895","3,055,196","3,055,196","3,055,841",0,5,1102,15412
6,TB4S000500,TVS Active Subs,2025-03-01 04:55:38,20240131,20250228,0,0,"1,220,146","1,219,989","1,219,989","1,307,223",0,5,8408,22760


In [7]:
''' METRIC Summary '''

src_t_rev_mth_df = chk_src_df.copy()
src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')


     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE  START_DAY   END_DAY                  P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-19 04:44:12   20240101  20250316                  0              5         1104   486864
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-19 04:41:15   20240101  20250316                  0              4          176    77616
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-19 04:42:48   20240101  20250316                  0              4          176    77616
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-19 04:50:14   20240131  20250317                  0              5         1104    16560
4   jutar11     TB1S000600   Prepay Active Caller Subs 2025-03-16 04:50:51   20240131  20250314        216,742,036              6         1105    16575
5   jutar11     TB2R000100            Postpaid Revenue 2025-03-19 04:20:10   20240102  

In [8]:
''' METRIC Current Month '''

# v_tm_key_mth = '202502'
curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {curr_mth}')

src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==curr_mth]
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202503

     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C              P              G              H             HH           CCAA CCAATT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-19 04:44:12     202503   20250301  20250316  0              0  1,332,304,559  1,332,298,669  1,332,298,669  1,326,661,283      0
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-19 04:41:15     202503   20250301  20250316  0              0    167,869,176    167,869,176    167,869,176              0      0
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-19 04:42:48     202503   20250301  20250316  0              0  1,164,435,384  1,164,429,493  1,164,429,493              0      0
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-19 04:50:14     202503   20250317  20250317  0              0     17,500,348     17,500,348     17,500,348     17,438,40

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [9]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB1R000100' --'Prepay Revenue
        , 'TB2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-03-20, 09:53:46

   -> Daily DataFrame : 146 rows, 12 columns

TDMDBPR : Disconnected


In [10]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 146 rows, 12 columns


In [11]:
''' TB1R000100 : Prepay Revenue '''

v_metric_cd = 'TB1R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB1R000100 : Prepay Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1R000100_day_df = chk_src_df.copy()
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['METRIC_CD']==v_metric_cd]
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1R000100_day_df = TB1R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1R000100_day_df[col] = TB1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1R000100_day_df


TB1R000100 : Prepay Revenue...

   -> v_metric_cd: TB1R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB1R000100,Prepay Revenue,20250301,2025-03-19 04:44:12,0,0,"98,894,369","98,893,385","98,893,385","98,438,050",0
1,jutar11,TB1R000100,Prepay Revenue,20250302,2025-03-19 04:44:12,0,0,"97,914,295","97,914,054","97,914,054","97,449,319",0
2,jutar11,TB1R000100,Prepay Revenue,20250303,2025-03-19 04:44:12,0,0,"97,339,883","97,339,357","97,339,357","96,896,679",0
3,jutar11,TB1R000100,Prepay Revenue,20250304,2025-03-19 04:44:12,0,0,"87,087,485","87,087,007","87,087,007","86,701,328",0
4,jutar11,TB1R000100,Prepay Revenue,20250305,2025-03-19 04:44:12,0,0,"88,603,471","88,603,314","88,603,314","88,249,841",0
5,jutar11,TB1R000100,Prepay Revenue,20250306,2025-03-19 04:44:12,0,0,"83,014,310","83,014,183","83,014,183","82,666,333",0
6,jutar11,TB1R000100,Prepay Revenue,20250307,2025-03-19 04:44:12,0,0,"84,159,378","84,158,146","84,158,146","83,814,588",0
7,jutar11,TB1R000100,Prepay Revenue,20250308,2025-03-19 04:44:12,0,0,"79,925,472","79,925,390","79,925,390","79,593,034",0
8,jutar11,TB1R000100,Prepay Revenue,20250309,2025-03-19 04:44:12,0,0,"77,198,685","77,198,635","77,198,635","76,883,895",0
9,jutar11,TB1R000100,Prepay Revenue,20250310,2025-03-19 04:44:12,0,0,"83,597,659","83,597,541","83,597,541","83,261,000",0


In [12]:
''' TB2R010100 : Postpaid Revenue B2C '''

v_metric_cd = 'TB2R010100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB2R010100 : Postpaid Revenue B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2R010100_day_df = chk_src_df.copy()
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['METRIC_CD']==v_metric_cd]
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2R010100_day_df = TB2R010100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2R010100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2R010100_day_df[col] = TB2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2R010100_day_df


TB2R010100 : Postpaid Revenue B2C...

   -> v_metric_cd: TB2R010100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB2R010100,Postpaid Revenue B2C,20250302,2025-03-19 04:21:08,0,"475,416,530","475,340,370","475,339,471","475,339,471","473,835,696",0
1,jutar11,TB2R010100,Postpaid Revenue B2C,20250306,2025-03-19 04:21:08,0,"279,723,176","279,699,259","279,699,259","279,699,259","278,381,419",0
2,jutar11,TB2R010100,Postpaid Revenue B2C,20250310,2025-03-19 04:21:08,0,"441,787,213","441,757,653","441,757,653","441,757,653","440,492,217",0
3,jutar11,TB2R010100,Postpaid Revenue B2C,20250313,2025-03-19 04:21:08,0,"388,864,445","388,823,480","388,823,031","388,823,031","387,680,118",0
4,jutar11,TB2R010100,Postpaid Revenue B2C,20250316,2025-03-19 04:21:08,0,"338,753,872","338,728,483","338,728,077","338,728,077","337,723,914",0


In [13]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250302,2025-03-19 05:10:10,0,0,"218,879,026","218,879,026","218,879,026","218,912,394",0
1,salisa,TB3R000100,Broadband Revenue,20250306,2025-03-19 05:10:10,0,0,"143,559,173","143,559,173","143,559,173","143,559,173",0
2,salisa,TB3R000100,Broadband Revenue,20250310,2025-03-19 05:10:10,0,0,"233,413,142","233,413,142","233,413,142","233,427,299",0
3,salisa,TB3R000100,Broadband Revenue,20250313,2025-03-19 05:10:10,0,0,"173,098,708","173,098,708","173,098,708","173,105,155",0
4,salisa,TB3R000100,Broadband Revenue,20250316,2025-03-19 05:10:10,0,0,"169,931,278","169,931,278","169,931,278","169,931,278",0


In [15]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB4R000100,TVS Revenue,20250302,2025-03-19 14:24:58,0,0,"33,915,217","33,928,404","33,945,767",0,0
1,salisa,TB4R000100,TVS Revenue,20250306,2025-03-19 14:24:58,0,0,"9,508,034","9,599,873","9,605,668",0,0
2,salisa,TB4R000100,TVS Revenue,20250310,2025-03-19 14:24:58,0,0,"29,561,765","29,671,805","29,689,081",0,0
3,salisa,TB4R000100,TVS Revenue,20250313,2025-03-19 14:24:58,0,0,"21,702,020","21,763,584","21,771,609",0,0
4,salisa,TB4R000100,TVS Revenue,20250316,2025-03-19 14:24:58,0,0,"22,913,596","22,913,596","22,913,596",0,0
